In [2]:


import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed and imported!")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [3]:
!pip install segmentation-models-pytorch
!pip install torchvision
!pip install opencv-python
!pip install nibabel
!pip install tqdm
!pip install scikit-learn

In [2]:
def prepare_kaggle_data():
    """Подготавливает DataFrame с путями к данным Kaggle"""
    file_list = []
    
    kaggle_dirs = [
        '/kaggle/input/liver-tumor-segmentation',
        '/kaggle/input/liver-tumor-segmentation-part-2',
        '../input/liver-tumor-segmentation',
        '../input/liver-tumor-segmentation-part-2'
    ]
    
    for base_dir in kaggle_dirs:
        if os.path.exists(base_dir):
            print(f"✅ Found directory: {base_dir}")
            for dirname, _, filenames in os.walk(base_dir):
                for filename in filenames:
                    if filename.endswith('.nii') or filename.endswith('.nii.gz'):
                        file_list.append((dirname, filename))
    
    if not file_list:
        print("❌ No NIfTI files found! Check directory paths.")
        return None
    
    df_files = pd.DataFrame(file_list, columns=['dirname', 'filename'])
    df_files = df_files.sort_values(by=['filename'], ascending=True)
    
    print(f"Total files found: {len(df_files)}")
    
    # Добавляем маски
    df_files["mask_dirname"] = ""
    df_files["mask_filename"] = ""
    
    matched_count = 0
    for i in range(131):
        ct = f"volume-{i}.nii"
        mask = f"segmentation-{i}.nii"
        
        ct_match = df_files['filename'] == ct
        if ct_match.any():
            df_files.loc[ct_match, 'mask_filename'] = mask
            # Пробуем разные пути для масок
            possible_mask_dirs = [
                "../input/liver-tumor-segmentation/segmentations",
                "/kaggle/input/liver-tumor-segmentation/segmentations",
                "../input/liver-tumor-segmentation",
                "/kaggle/input/liver-tumor-segmentation"
            ]
            for mask_dir in possible_mask_dirs:
                if os.path.exists(mask_dir):
                    df_files.loc[ct_match, 'mask_dirname'] = mask_dir
                    break
            matched_count += 1
    
    print(f"Matched {matched_count} volume-mask pairs")
    
    # Фильтруем и добавляем пути
    df_volumes_with_masks = df_files[df_files['mask_filename'] != ''].copy()
    df_volumes_with_masks = df_volumes_with_masks.reset_index(drop=True)
    
    df_volumes_with_masks['ct_path'] = df_volumes_with_masks['dirname'] + '/' + df_volumes_with_masks['filename']
    df_volumes_with_masks['mask_path'] = df_volumes_with_masks['mask_dirname'] + '/' + df_volumes_with_masks['mask_filename']
    
    # Проверяем существование файлов
    existing_files = 0
    for idx, row in df_volumes_with_masks.iterrows():
        if os.path.exists(row['ct_path']) and os.path.exists(row['mask_path']):
            existing_files += 1
    
    print(f"Final dataset: {len(df_volumes_with_masks)} volumes with masks")
    print(f"Files that exist: {existing_files}/{len(df_volumes_with_masks)}")
    
    return df_volumes_with_masks

# Запускаем подготовку данных
print("🔧 Preparing Kaggle data...")
df_kaggle = prepare_kaggle_data()

if df_kaggle is not None:
    print("✅ Kaggle data loaded successfully!")
    df_kaggle.head()
else:
    print("❌ Failed to load Kaggle data")

🔧 Preparing Kaggle data...
✅ Found directory: /kaggle/input/liver-tumor-segmentation
✅ Found directory: /kaggle/input/liver-tumor-segmentation-part-2
✅ Found directory: ../input/liver-tumor-segmentation
✅ Found directory: ../input/liver-tumor-segmentation-part-2
Total files found: 524
Matched 131 volume-mask pairs
Final dataset: 262 volumes with masks
Files that exist: 262/262
✅ Kaggle data loaded successfully!


In [3]:
def read_nii(filepath):
    """Читает .nii файл и возвращает numpy array"""
    ct_scan = nib.load(filepath)
    array = ct_scan.get_fdata()
    array = np.rot90(np.array(array))
    return array

def preprocess_and_save_data(df_files, output_dir='processed_data_deeplab_256', image_size=256):
    """
    Предобрабатывает данные и сохраняет на диск для DeepLabV3+
    """
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f'{output_dir}/images', exist_ok=True)
    os.makedirs(f'{output_dir}/masks', exist_ok=True)
    
    image_paths = []
    mask_paths = []
    volume_ids = []
    slice_ids = []
    
    print(f"🚀 Starting preprocessing for DeepLabV3+ - saving to {output_dir}")
    print(f"📐 Target size: {image_size}x{image_size}")
    
    for idx in tqdm(range(len(df_files)), desc="Processing volumes"):
        try:
            # Проверяем существование файлов
            ct_path = df_files.loc[idx, 'ct_path']
            mask_path = df_files.loc[idx, 'mask_path']
            
            if not os.path.exists(ct_path) or not os.path.exists(mask_path):
                print(f"⚠️ Files not found for volume {idx}")
                continue
            
            # Чтение данных
            ct_scan = read_nii(ct_path)
            mask = read_nii(mask_path)
            
            # Получаем ID volume из имени файла
            file_name = df_files.loc[idx, 'filename']
            volume_id = file_name.replace('volume-', '').replace('.nii', '')
            
            # Обрабатываем ВСЕ срезы
            num_slices = ct_scan.shape[2]
            
            for slice_idx in range(num_slices):
                # Извлекаем срез
                ct_slice = ct_scan[..., slice_idx]
                mask_slice = mask[..., slice_idx]
                
                # Нормализация CT среза [0, 1]
                ct_slice_normalized = (ct_slice - ct_slice.min()) / (ct_slice.max() - ct_slice.min() + 1e-8)
                
                # Конвертируем в uint8 [0, 255] для сохранения
                ct_slice_uint8 = (ct_slice_normalized * 255).astype(np.uint8)
                mask_slice_uint8 = mask_slice.astype(np.uint8)
                
                # Изменение размера
                ct_slice_resized = cv2.resize(ct_slice_uint8, (image_size, image_size))
                mask_slice_resized = cv2.resize(mask_slice_uint8, (image_size, image_size), 
                                               interpolation=cv2.INTER_NEAREST)
                
                # Сохранение изображений
                image_filename = f'volume_{volume_id}_slice_{slice_idx:03d}.png'
                mask_filename = f'volume_{volume_id}_slice_{slice_idx:03d}_mask.png'
                
                image_path = f'{output_dir}/images/{image_filename}'
                mask_path = f'{output_dir}/masks/{mask_filename}'
                
                # Сохраняем как PNG
                cv2.imwrite(image_path, ct_slice_resized)
                cv2.imwrite(mask_path, mask_slice_resized)
                
                # Сохраняем пути
                image_paths.append(image_path)
                mask_paths.append(mask_path)
                volume_ids.append(int(volume_id))
                slice_ids.append(slice_idx)
                
        except Exception as e:
            print(f"❌ Error processing volume {idx}: {e}")
            continue
    
    # Создаем DataFrame с путями
    df_processed = pd.DataFrame({
        'image_path': image_paths,
        'mask_path': mask_paths,
        'volume_id': volume_ids,
        'slice_id': slice_ids
    })
    
    # Сохраняем CSV с путями
    csv_path = f'{output_dir}/dataset.csv'
    df_processed.to_csv(csv_path, index=False)
    
    print(f"✅ Preprocessing completed!")
    print(f"📁 Saved {len(df_processed)} image-mask pairs")
    print(f"💾 CSV saved: {csv_path}")
    
    return df_processed

# Запускаем предобработку
if df_kaggle is not None:
    print("🔄 Starting data preprocessing for DeepLabV3+...")
    df_processed = preprocess_and_save_data(df_kaggle, image_size=256)
    print("✅ Data preprocessing completed!")
    df_processed.head()
else:
    print("❌ Cannot preprocess data - no Kaggle data loaded")

🔄 Starting data preprocessing for DeepLabV3+...
🚀 Starting preprocessing for DeepLabV3+ - saving to processed_data_deeplab_256
📐 Target size: 256x256


Processing volumes:   0%|          | 0/262 [00:00<?, ?it/s]

✅ Preprocessing completed!
📁 Saved 117276 image-mask pairs
💾 CSV saved: processed_data_deeplab_256/dataset.csv
✅ Data preprocessing completed!


In [6]:
import segmentation_models_pytorch as smp

# DeepLabV3+ Model Class
class DeepLabV3PlusModel:
    def __init__(self, num_classes=3, encoder_name='resnet50', encoder_weights='imagenet'):
        self.model = smp.DeepLabV3Plus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            classes=num_classes,
            activation=None,
            in_channels=1  # 1 канал для CT снимков
        )
    
    def to(self, device):
        self.model = self.model.to(device)
        return self
    
    def train(self):
        self.model.train()
    
    def eval(self):
        self.model.eval()
    
    def parameters(self):
        return self.model.parameters()
    
    def state_dict(self):
        return self.model.state_dict()
    
    def load_state_dict(self, state_dict):
        self.model.load_state_dict(state_dict)

print("🧠 Initializing DeepLabV3+ model...")
deeplab_model = DeepLabV3PlusModel(num_classes=3, encoder_name='resnet50')
print("✅ DeepLabV3+ model created!")

🧠 Initializing DeepLabV3+ model...


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

✅ DeepLabV3+ model created!


In [1]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
class LiverDatasetDeepLab(Dataset):
    def __init__(self, df, image_size=256, transform=None):
        self.df = df
        self.image_size = image_size
        self.transform = transform
        self.image_paths = df['image_path'].values
        self.mask_paths = df['mask_path'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Загрузка готовых PNG
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        
        # Загрузка изображений
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Нормализация [0, 255] -> [0, 1]
        image = image.astype(np.float32) / 255.0
        
        # Добавляем канальное измерение для DeepLab
        image = np.expand_dims(image, axis=0)  # [1, H, W]
        
        # Преобразование в тензоры
        image_tensor = torch.FloatTensor(image)  # [1, H, W]
        mask_tensor = torch.LongTensor(mask)     # [H, W]
        
        return image_tensor, mask_tensor

def create_train_val_test_datasets(df_processed, image_size=256):
    """Создает train, validation и test датасеты"""
    
    print(f"📊 Creating train/val/test datasets from {len(df_processed)} samples")
    
    # Сначала разделяем на train+val (85%) и test (15%)
    train_val_df, test_df = train_test_split(
        df_processed, test_size=0.15, random_state=42, shuffle=True
    )
    
    # Затем разделяем train+val на train (80%) и val (20%)
    train_df, val_df = train_test_split(
        train_val_df, test_size=0.2, random_state=42, shuffle=True
    )
    
    print(f"📊 Training samples: {len(train_df)}")
    print(f"📊 Validation samples: {len(val_df)}")
    print(f"📊 Test samples: {len(test_df)}")
    
    # Создаем датасеты
    train_dataset = LiverDatasetDeepLab(train_df, image_size=image_size)
    val_dataset = LiverDatasetDeepLab(val_df, image_size=image_size)
    test_dataset = LiverDatasetDeepLab(test_df, image_size=image_size)
    
    return train_dataset, val_dataset, test_dataset, train_df, val_df, test_df

# Создание датасетов
print("🔄 Creating train/val/test datasets...")
train_dataset_deeplab, val_dataset_deeplab, test_dataset_deeplab, train_df_deeplab, val_df_deeplab, test_df_deeplab = create_train_val_test_datasets(df_processed)

print("✅ Datasets created!")

def create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=8):
    """Создает DataLoader'ы"""
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True
    )
    
    print(f"📊 DataLoaders created:")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Training batches: {len(train_loader)}")
    print(f"  - Validation batches: {len(val_loader)}")
    print(f"  - Test batches: {len(test_loader)}")
    
    return train_loader, val_loader, test_loader

# Создание DataLoader'ов
print("🔄 Creating DataLoaders...")
train_loader_deeplab, val_loader_deeplab, test_loader_deeplab = create_dataloaders(
    train_dataset_deeplab, val_dataset_deeplab, test_dataset_deeplab, batch_size=8
)
print("✅ DataLoaders ready!")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [9]:
def create_dataset_csv(processed_dir='processed_data_deeplab_256'):
    """
    Создает CSV файл с путями к уже обработанным данным
    """
    images_dir = os.path.join(processed_dir, 'images')
    masks_dir = os.path.join(processed_dir, 'masks')
    
    # Проверяем существование директорий
    if not os.path.exists(images_dir) or not os.path.exists(masks_dir):
        print(f"❌ Processed directories not found: {images_dir} or {masks_dir}")
        return None
    
    image_paths = []
    mask_paths = []
    volume_ids = []
    slice_ids = []
    
    print(f"🔍 Scanning existing processed data in {processed_dir}")
    
    # Собираем все файлы изображений
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.png') and not f.endswith('_mask.png')]
    
    for image_file in tqdm(image_files, desc="Processing existing files"):
        try:
            # Парсим имя файла для получения volume_id и slice_id
            # Формат: volume_{volume_id}_slice_{slice_idx}.png
            base_name = image_file.replace('.png', '')
            parts = base_name.split('_')
            
            if len(parts) >= 4 and parts[0] == 'volume' and parts[2] == 'slice':
                volume_id = parts[1]
                slice_id = parts[3]
                
                # Формируем пути
                image_path = os.path.join(images_dir, image_file)
                mask_filename = f'volume_{volume_id}_slice_{slice_id}_mask.png'
                mask_path = os.path.join(masks_dir, mask_filename)
                
                # Проверяем существование маски
                if os.path.exists(mask_path):
                    image_paths.append(image_path)
                    mask_paths.append(mask_path)
                    volume_ids.append(int(volume_id))
                    slice_ids.append(int(slice_id))
                else:
                    print(f"⚠️ Mask not found: {mask_filename}")
                    
        except Exception as e:
            print(f"❌ Error processing file {image_file}: {e}")
            continue
    
    # Создаем DataFrame
    df_processed = pd.DataFrame({
        'image_path': image_paths,
        'mask_path': mask_paths,
        'volume_id': volume_ids,
        'slice_id': slice_ids
    })
    
    # Сохраняем CSV
    csv_path = os.path.join(processed_dir, 'dataset.csv')
    df_processed.to_csv(csv_path, index=False)
    
    print(f"✅ Dataset CSV created!")
    print(f"📊 Found {len(df_processed)} image-mask pairs")
    print(f"💾 CSV saved: {csv_path}")
    
    return df_processed

# Альтернативная версия - просто загружает существующий CSV если он есть
def load_existing_dataset(processed_dir='processed_data_deeplab_256'):
    """
    Загружает существующий датасет из CSV файла
    """
    csv_path = os.path.join(processed_dir, 'dataset.csv')
    
    if os.path.exists(csv_path):
        print(f"📁 Loading existing dataset from {csv_path}")
        df_processed = pd.read_csv(csv_path)
        print(f"✅ Loaded {len(df_processed)} image-mask pairs")
        return df_processed
    else:
        print(f"📁 CSV not found, creating new one...")
        return create_dataset_csv(processed_dir)

# Проверяем и загружаем данные
print("🔄 Checking for existing processed data...")

# Попробуем загрузить существующие данные
df_processed = load_existing_dataset('processed_data_deeplab_256')

if df_processed is not None and len(df_processed) > 0:
    print("✅ Successfully loaded processed dataset!")
    print(f"📊 Dataset info: {len(df_processed)} samples")
    print(df_processed.head())
else:
    print("❌ No processed data found. You need to run preprocessing first.")
    print("To preprocess data, run:")
    print("df_processed = preprocess_and_save_data(df_kaggle, image_size=256)")

🔄 Checking for existing processed data...
📁 Loading existing dataset from processed_data_deeplab_256/dataset.csv
✅ Loaded 117276 image-mask pairs
✅ Successfully loaded processed dataset!
📊 Dataset info: 117276 samples
                                          image_path  \
0  processed_data_deeplab_256/images/volume_0_sli...   
1  processed_data_deeplab_256/images/volume_0_sli...   
2  processed_data_deeplab_256/images/volume_0_sli...   
3  processed_data_deeplab_256/images/volume_0_sli...   
4  processed_data_deeplab_256/images/volume_0_sli...   

                                           mask_path  volume_id  slice_id  
0  processed_data_deeplab_256/masks/volume_0_slic...          0         0  
1  processed_data_deeplab_256/masks/volume_0_slic...          0         1  
2  processed_data_deeplab_256/masks/volume_0_slic...          0         2  
3  processed_data_deeplab_256/masks/volume_0_slic...          0         3  
4  processed_data_deeplab_256/masks/volume_0_slic...          0  

In [10]:
def calculate_metrics(preds, targets, num_classes=3):
    """Вычисление метрик: IoU и Dice для каждого класса"""
    ious = []
    dices = []
    preds = torch.argmax(preds, dim=1)
    
    for cls in range(num_classes):
        pred_inds = (preds == cls)
        target_inds = (targets == cls)
        
        intersection = (pred_inds & target_inds).float().sum()
        union = (pred_inds | target_inds).float().sum()
        
        # IoU
        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(float(intersection / union))
        
        # Dice
        if (pred_inds.float().sum() + target_inds.float().sum()) == 0:
            dices.append(float('nan'))
        else:
            dice = (2. * intersection) / (pred_inds.float().sum() + target_inds.float().sum())
            dices.append(float(dice))
    
    return np.nanmean(ious), np.nanmean(dices)

def train_deeplabv3plus(model, train_loader, val_loader, num_epochs=15, device='cuda'):
    """Обучение DeepLabV3+ модели"""
    
    from torch.optim import Adam
    from torch.optim.lr_scheduler import ReduceLROnPlateau
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)
    
    model.to(device)
    
    # Для отслеживания метрик
    train_losses = []
    val_losses = []
    val_ious = []
    val_dices = []
    
    best_iou = 0.0
    best_dice = 0.0
    
    print(f"🚀 Starting DeepLabV3+ training for {num_epochs} epochs...")
    print(f"📊 Training on {len(train_loader.dataset)} samples")
    print(f"📊 Validating on {len(val_loader.dataset)} samples")
    print("=" * 60)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        start_time = time.time()
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch_idx, (images, masks) in enumerate(train_pbar):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model.model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        epoch_val_ious = []
        epoch_val_dices = []
        
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
        with torch.no_grad():
            for images, masks in val_pbar:
                images, masks = images.to(device), masks.to(device)
                outputs = model.model(images)
                
                loss = criterion(outputs, masks)
                val_loss += loss.item()
                
                iou, dice = calculate_metrics(outputs, masks)
                epoch_val_ious.append(iou)
                epoch_val_dices.append(dice)
                
                val_pbar.set_postfix({
                    'Loss': f'{loss.item():.4f}', 
                    'mIoU': f'{iou:.4f}',
                    'Dice': f'{dice:.4f}'
                })
        
        # Calculate epoch metrics
        epoch_train_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_iou = np.mean(epoch_val_ious)
        epoch_val_dice = np.mean(epoch_val_dices)
        epoch_time = time.time() - start_time
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        val_ious.append(epoch_val_iou)
        val_dices.append(epoch_val_dice)
        
        scheduler.step(epoch_val_loss)
        
        # Print epoch results
        print(f'\n✅ Epoch {epoch+1} Complete:')
        print(f'   ⏱️  Time: {epoch_time/60:.1f} min')
        print(f'   📉 Train Loss: {epoch_train_loss:.4f}')
        print(f'   📊 Val Loss: {epoch_val_loss:.4f}')
        print(f'   🎯 mIoU: {epoch_val_iou:.4f}')
        print(f'   🎯 Dice: {epoch_val_dice:.4f}')
        print(f'   📚 LR: {optimizer.param_groups[0]["lr"]:.2e}')
        
        # Save best model
        if epoch_val_iou > best_iou:
            best_iou = epoch_val_iou
            best_dice = epoch_val_dice
            torch.save(model.state_dict(), 'best_deeplabv3plus_liver.pth')
            print(f'   💾 Best model saved! (IoU: {best_iou:.4f}, Dice: {best_dice:.4f})')
        
        print('-' * 50)
    
    # Save final model
    torch.save(model.state_dict(), 'final_deeplabv3plus_liver.pth')
    print(f"🎉 Training completed! Best IoU: {best_iou:.4f}")
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_ious': val_ious,
        'val_dices': val_dices,
        'best_iou': best_iou,
        'best_dice': best_dice
    }

NameError: name 'df_kaggle' is not defined

In [ ]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name()}")
    print(f"🎯 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Запуск обучения
print("🔥 Starting DeepLabV3+ Training...")
print("=" * 50)

results_deeplab = train_deeplabv3plus(
    model=deeplab_model,
    train_loader=train_loader_deeplab,
    val_loader=val_loader_deeplab,
    num_epochs=15,
    device=device
)

print("🎉 DeepLabV3+ training completed!")

🖥️ Using device: cuda
🎯 GPU: Tesla T4
🎯 GPU Memory: 14.7 GB
🔥 Starting DeepLabV3+ Training...
🚀 Starting DeepLabV3+ training for 15 epochs...
📊 Training on 79747 samples
📊 Validating on 19937 samples


Epoch 1/15 [Train]:   0%|          | 0/9969 [00:00<?, ?it/s]

In [ ]:
def plot_training_results(results):
    """Визуализация результатов обучения"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(results['train_losses'], label='Train Loss', linewidth=2, color='blue')
    axes[0, 0].plot(results['val_losses'], label='Val Loss', linewidth=2, color='red')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # mIoU
    axes[0, 1].plot(results['val_ious'], label='mIoU', linewidth=2, color='green')
    axes[0, 1].set_title('Validation mIoU')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('mIoU')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Dice
    axes[1, 0].plot(results['val_dices'], label='Dice', linewidth=2, color='orange')
    axes[1, 0].set_title('Validation Dice Coefficient')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Dice')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Combined metrics
    axes[1, 1].plot(results['val_ious'], label='mIoU', linewidth=2, color='green')
    axes[1, 1].plot(results['val_dices'], label='Dice', linewidth=2, color='orange')
    axes[1, 1].set_title('Validation Metrics Comparison')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print final results
    print(f"\n🏆 FINAL DEEPLABV3+ RESULTS:")
    print("=" * 40)
    print(f"Best mIoU: {results['best_iou']:.4f}")
    print(f"Best Dice: {results['best_dice']:.4f}")
    print(f"Final mIoU: {results['val_ious'][-1]:.4f}")
    print(f"Final Dice: {results['val_dices'][-1]:.4f}")

# Визуализация результатов
print("📊 Plotting training results...")
plot_training_results(results_deeplab)